In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import owlready2 as owl
from owlready2 import *
owlready2.reasoning.JAVA_MEMORY = 20000

import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

torch.manual_seed(33)

In [ ]:
from create_pytorch_dataset import GeometricInterpretation, CanonicalModel, EntityEmbedding, SCALE_FACTOR
from create_pytorch_dataset import entity_to_idx_vocab, concept_to_idx_vocab, role_to_idx_vocab, idx_to_entity_vocab, idx_to_concept_vocab, idx_to_role_vocab, idx_finder_dict
from create_pytorch_dataset import trainConceptDataset, testConceptDataset, trainRoleDataset, testRoleDataset, train_ConceptDataLoader, test_ConceptDataLoader, train_RoleDataLoader, test_RoleDataLoader, RoleDataLoader, ConceptDataLoader

In [ ]:
from FaithEL_model import FaithEL
from utils import save_model, train_model, plot_score_hak, plot_model

In [ ]:
CENTROID_SCORE = True # When set to True, model scores assertion w.r.t distance to the centroid and to the moving parameter for concepts/roles
LR = 0.001
PHI = 1 # Weighs the loss obtained by computing the distance between the role/concept params to the centroid
GAMMA = 1 # Weighs the loss obtained by computing the distance between the role/concept param to the individual/ind_concat parameters
PSI = 0.7 # Weights the negative loss obtained by computing the distance between the real individual/ind_concat and neg sampled individuals/ind_concats
RADIUS = SCALE_FACTOR/2 + 0.1
EMB_DIM = 602

LOG_EPOCH = 10000
EVAL_TRAIN = False
EPOCHS = 200
EVAL_FREQ = 1

NEG_SAMPLING = True
PLOT_LOSS = True

TRAINING_CATEGORY = 0 # 0 = both, 1 = concept only, 2 = role only

DIM1 = 0
DIM2 = 1

loss_fn = nn.MSELoss()

In [ ]:
torch.manual_seed(269)

model = FaithEL(EMB_DIM, PHI, RADIUS, GAMMA, PSI,
                entity_to_idx_vocab, concept_to_idx_vocab, role_to_idx_vocab,
                )

optimizer = optim.Adam(model.parameters(), lr=LR)

In [ ]:
model_list = []

In [ ]:
train_loss_list, test_loss_list, train_hits_at_k_concept, test_hits_at_k_concept, train_hits_at_k_role, test_hits_at_k_role, = train_model(model, GeometricInterpretation,
                                                                                                                                           train_ConceptDataLoader, train_RoleDataLoader, test_ConceptDataLoader, test_RoleDataLoader,
                                                                                                                                           trainConceptDataset, testConceptDataset, trainRoleDataset, testRoleDataset,
                                                                                                                                           EPOCHS, LOG_EPOCH, EVAL_FREQ, EVAL_TRAIN, loss_fn, optimizer,
                                                                                                                                           idx_to_entity_vocab, entity_to_idx_vocab,
                                                                                                                                           idx_to_concept_vocab, concept_to_idx_vocab,
                                                                                                                                           idx_to_role_vocab, role_to_idx_vocab,
                                                                                                                                           CENTROID_SCORE, NEG_SAMPLING, PLOT_LOSS, TRAINING_CATEGORY
                                                                                                                                           )

model_list.append(save_model(CENTROID_SCORE, LR, PHI, GAMMA, PSI, EMB_DIM, EPOCHS, LOG_EPOCH, EVAL_FREQ, EVAL_TRAIN,
                             loss_fn, model, optimizer, train_loss_list, test_loss_list, train_hits_at_k_concept, test_hits_at_k_concept, train_hits_at_k_role, test_hits_at_k_role))

In [ ]:
pos_list = -1
TOPK = 2

In [ ]:
plot_score_hak(model_list[pos_list]['test_hits_at_k_concept'], model_list[pos_list]['test_hits_at_k_role'], TOPK, model_list[pos_list]['epochs'], model_list[pos_list]['eval_freq'])
# Uncomment if evaluating on training data
# plot_score_hak(model_list[pos_list]['train_hits_at_k_concept'], model_list[pos_list]['train_hits_at_k_role'], TOPK, model_list[pos_list]['epochs'], model_list[pos_list]['eval_freq'])